In [25]:
from __future__ import annotations
import os, sys
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import repeat
from multiprocessing import cpu_count, freeze_support
from concurrent.futures import ProcessPoolExecutor
import json


In [26]:
# 1. 定义 CSV 文件路径
user_csv_path = r'H:\nighttimeLights\south_2017tif_layers\cleaned_pixel_timeseries_avg_gte5.csv'

# 2. 加载 CSV 文件
try:
    print(f"尝试从以下路径加载 CSV 文件: {user_csv_path}")
    raw_df = pd.read_csv(user_csv_path)
    print(f"成功加载 CSV. 文件形状 (Shape): {raw_df.shape}")
    print(f"列名: {raw_df.columns.tolist()}")
except FileNotFoundError:
    print(f"错误：文件 '{user_csv_path}' 未找到。请检查路径。")
    sys.exit(1) # 如果文件未找到，则退出脚本
except Exception as e:
    print(f"加载 CSV 时发生错误: {e}")
    sys.exit(1)

尝试从以下路径加载 CSV 文件: H:\nighttimeLights\south_2017tif_layers\cleaned_pixel_timeseries_avg_gte5.csv
成功加载 CSV. 文件形状 (Shape): (82, 214)
列名: ['X', 'Y', '2017152', '2017153', '2017154', '2017155', '2017156', '2017157', '2017158', '2017159', '2017160', '2017161', '2017162', '2017163', '2017164', '2017165', '2017166', '2017167', '2017168', '2017169', '2017170', '2017171', '2017172', '2017173', '2017174', '2017175', '2017176', '2017177', '2017178', '2017179', '2017180', '2017181', '2017182', '2017183', '2017184', '2017185', '2017186', '2017187', '2017188', '2017189', '2017190', '2017191', '2017192', '2017193', '2017194', '2017195', '2017196', '2017197', '2017198', '2017199', '2017200', '2017201', '2017202', '2017203', '2017204', '2017205', '2017206', '2017207', '2017208', '2017209', '2017210', '2017211', '2017212', '2017213', '2017214', '2017215', '2017216', '2017217', '2017218', '2017219', '2017220', '2017221', '2017222', '2017223', '2017224', '2017225', '2017226', '2017227', '2017228', '2017229

In [27]:
# 3. 识别 X 和 Y 坐标列
if len(raw_df.columns) < 2:
    print("错误：CSV 文件至少需要两列作为 X 和 Y 坐标。")
    sys.exit(1)
    
x_column_name = raw_df.columns[0]
y_column_name = raw_df.columns[1]
print(f"识别到的 X 列: '{x_column_name}', Y 列: '{y_column_name}'")

# 4. 为每一行创建一个唯一的 ID 作为索引
raw_df['unique_id'] = raw_df[x_column_name].astype(str) + "_" + raw_df[y_column_name].astype(str)
print("已创建 'unique_id' 列。")

# 5. 将新的 'unique_id' 设置为数据框的索引
df_indexed = raw_df.set_index('unique_id')
print(f"已将 'unique_id' 设置为索引。索引名称: {df_indexed.index.name}")

# 6. 仅选择时间序列数据列
# 这些是除了原始 X 和 Y 坐标列之外的所有列。
time_series_column_names = [col for col in df_indexed.columns if col not in [x_column_name, y_column_name]]

if not time_series_column_names:
    print("错误：排除 X, Y 和 unique_id 后，未找到时间序列数据列。")
    sys.exit(1)
    
print(f"识别到 {len(time_series_column_names)} 个时间序列列。")
cleaned_wide_data_avrg = df_indexed[time_series_column_names].copy() # 使用 .copy() 避免 SettingWithCopyWarning

# 7. 确保时间序列值为数值类型
# BEAST 分析（以及 MAD 异常值移除）期望输入的是数值型（浮点型）数据。
print("正在将时间序列数据转换为数值型 (float)...")
try:
    # 尝试将所有选定的时间序列列转换为 float
    for col in cleaned_wide_data_avrg.columns:
        cleaned_wide_data_avrg[col] = pd.to_numeric(cleaned_wide_data_avrg[col], errors='raise') # errors='raise' 会在无法转换时报错
    cleaned_wide_data_avrg = cleaned_wide_data_avrg.astype(float)
    print("时间序列数据成功转换为 float 类型。")
except ValueError as e:
    print(f"警告：无法将所有时间序列数据转换为 float。错误信息: {e}")
    print("请检查你的 CSV 文件中数据列（除了前两列）是否包含非数值。")
    print("BEAST 算法需要数值型的时间序列数据。")
    # 你可能需要检查是哪个列/值导致了错误，或者直接 sys.exit(1)



print(f"准备好进行处理的 'cleaned_wide_data_avrg' 的最终形状: {cleaned_wide_data_avrg.shape}")
if not cleaned_wide_data_avrg.empty:
    print(f"'cleaned_wide_data_avrg' 的索引: {cleaned_wide_data_avrg.index.name}, 示例: {cleaned_wide_data_avrg.index[0]}")
    print(f"'cleaned_wide_data_avrg' 的列名 (前5个): {cleaned_wide_data_avrg.columns.tolist()[:5]}...")
else:
    print("警告: 'cleaned_wide_data_avrg' 在开始异常值移除前为空。请检查之前的步骤。")

# --- 针对你的 CSV 文件的修改结束 ---

# 你的脚本余下部分从这里继续：
# 例如，remove_outliers_mad 函数的定义以及后续的处理。
# 确保 'cleaned_wide_data_avrg' 已按脚本预期正确准备好。

# 例如，你的下一部分代码可能是：
# def remove_outliers_mad( ... ):
# ...
# cleaned_wide_data_avrg = remove_outliers_mad(cleaned_wide_data_avrg, k=5, min_valid=3, max_offset=3)
# ... 等等

识别到的 X 列: 'X', Y 列: 'Y'
已创建 'unique_id' 列。
已将 'unique_id' 设置为索引。索引名称: unique_id
识别到 212 个时间序列列。
正在将时间序列数据转换为数值型 (float)...
时间序列数据成功转换为 float 类型。
准备好进行处理的 'cleaned_wide_data_avrg' 的最终形状: (82, 212)
'cleaned_wide_data_avrg' 的索引: unique_id, 示例: -81.80416667_26.3
'cleaned_wide_data_avrg' 的列名 (前5个): ['2017152', '2017153', '2017154', '2017155', '2017156']...


In [28]:
#对异常值进行清理
def remove_outliers_mad(
    df, 
    k=5, 
    min_valid=3, 
    max_offset=3,
    verbose=False
):
    """
    移除时间序列中的局部异常值（基于中位数 + MAD）
    
    参数:
    - df: pd.DataFrame，每列为时间（应为datetime格式），每行为一个序列
    - k: 异常值判定的阈值倍数，默认5
    - min_valid: 最少需要几个邻近有效值才能判断异常
    - max_offset: 最多从 t 向前向后扩展几天以寻找邻近值
    - verbose: 如果为True，会打印一些检测信息（默认False）
    
    返回:
    - 一个清洗后的新DataFrame（异常值设为NaN）
    """

    df_cleaned = df.copy()

    for i, row_idx in enumerate(df.index):
        series = df.loc[row_idx]
        for t_idx in range(len(series)):
            neighbors = []
            offset = 1

            while offset <= max_offset and len(neighbors) < min_valid:
                for delta in [-offset, offset]:
                    neighbor_idx = t_idx + delta
                    if 0 <= neighbor_idx < len(series):
                        val = series.iloc[neighbor_idx]
                        if pd.notna(val):
                            neighbors.append(val)
                offset += 1

            if len(neighbors) >= min_valid and pd.notna(series.iloc[t_idx]):
                median = np.median(neighbors)
                mad = np.median([abs(x - median) for x in neighbors])
                if mad > 0 and abs(series.iloc[t_idx] - median) > k * mad:
                    df_cleaned.iat[i, t_idx] = np.nan
                    if verbose:
                        print(f"Row {row_idx}, Column {df.columns[t_idx]} marked as outlier: {series.iloc[t_idx]}")

    return df_cleaned

cleaned_wide_data_avrg = remove_outliers_mad(cleaned_wide_data_avrg, k=5, min_valid=3, max_offset=3)

In [ ]:

#  1. 备份数据
if 'cleaned_wide_data_avrg' not in globals():
    print("！错误：在当前环境未找到 cleaned_wide_data_avrg，请先读取")
    print("cleaned_wide_data_avrg = pd.read_csv(..., index_col=0)")
    sys.exit(1)

df = cleaned_wide_data_avrg


# 把列名转成 DatetimeIndex，并排序
# !!! 下面是需要修改的关键行 !!!
try:
    print("尝试将列名转换为日期时间对象...")
    print(f"转换前的列名 (前5个): {df.columns.tolist()[:5]}")
    df.columns = pd.to_datetime(df.columns, format='%Y%j') # <--- 修改这里
    print(f"转换后的列名 (前5个，类型 {type(df.columns)}): {df.columns.tolist()[:5]}")
except ValueError as e:
    print(f"错误：转换列名为日期时间时失败。请检查列名是否都符合 'YYYYDOY' (例如 '2017152') 格式。")
    print(f"具体错误信息: {e}")
    sys.exit(1)
except Exception as e:
    print(f"转换列名时发生未知错误: {e}")
    sys.exit(1)




df = df.sort_index(axis=1)
dates = df.columns
freq  = 1
mask_num = 0.1
season_num = None
formula_type = 'y ~ trend' # or 'y ~ trend + season'

json_out_path = r"H:\nighttimeLights\south_2017tif_layers\beast\beast_model_outputs_12021.json"
out_trend_path = r"H:\nighttimeLights\south_2017tif_layers\beast\beast_trend_12021.csv"
out_cp_path    = r"H:\nighttimeLights\south_2017tif_layers\beast\beast_change_points_12021.csv"

results = []
cp_records = []

# —— 2. BEAST 单序列函数 —— 
def extract_cp_info(res):
    import numpy as np

    try:
        cp_raw = getattr(res.trend, "cp", None)
        print(f"[DEBUG] raw cp = {cp_raw} ({type(cp_raw)})")

        # 转换为 ndarray
        cp_array = np.asarray(cp_raw).ravel()
        print(f"[DEBUG] cp_array (raveled) = {cp_array}")

        # 去除 NaN
        cp_array_clean = cp_array[~np.isnan(cp_array)]
        print(f"[DEBUG] cp_array_clean = {cp_array_clean}")

        # 转换为 int index
        cp_index = cp_array_clean.astype(int)
        print(f"[DEBUG] cp_index = {cp_index}")

        # 再提取概率
        cp_prob_full = getattr(res.trend, "cpOccPr", None)
        cp_prob_array = np.asarray(cp_prob_full).ravel()
        print(f"[DEBUG] cpOccPr full = {cp_prob_array} (len={len(cp_prob_array)})")

        if cp_index.size > 0 and cp_index.max() < len(cp_prob_array):
            cp_prob = cp_prob_array[cp_index]
        else:
            cp_prob = np.array([], dtype=float)

        print(f"[OK] extracted cp_index = {cp_index}, cp_prob = {cp_prob}")
        return cp_index, cp_prob

    except Exception as e:
        print("[ERROR in extract_cp_info]:", e)
        return np.array([], dtype=int), np.array([], dtype=float)



def run_beast_on_series(h3_id, values, dates, freq=1):
    import Rbeast, numpy as np, pandas as pd
    


    ts = pd.Series(values.astype(float), index=dates)

    res = Rbeast.beast(
        ts,
        freq=freq,
        season=season_num,
        what='all',
        formula=formula_type,
        extra={
            "computeCredible": True,
            "computeTrendChngpt": True,
            "computeTrendSlope": True,
            "computeSeasonOrder": True,
            "computeTrendOrder": True,
            "computeSeasonChngpt": True,
            "quiet": True,
            "printProgress": False,
            "dumpInputData": False
        }
    )


    def _to_list(comp):
        import numpy as np, collections.abc as cabc
        if isinstance(comp, np.ndarray):
            return comp.tolist()
        for attr in ("est", "value", "mean", "Y", "Yhat"):  # 加上 Yhat
            if hasattr(comp, attr):
                return np.asarray(getattr(comp, attr)).squeeze().tolist()
        if isinstance(comp, (list, tuple, cabc.Iterable)):
            return list(comp)
        raise TypeError(f"Unrecognized BEAST component: {type(comp)}")

    trend_vals  = _to_list(res.trend)
    fitted_vals = _to_list(res.fitted) if hasattr(res, "fitted") else None

    # 过滤显著突变点
    cp_index, cp_prob = extract_cp_info(res)
    print(f"[{h3_id}] After extract_cp_info → cp_index: {cp_index}, cp_prob: {cp_prob}")
    assert cp_prob.size in (0, cp_index.size), \
       f"extract_cp_info mismatch: cp={cp_index.size}, prob={cp_prob.size}"
    
    if cp_index.size and cp_prob.size and cp_prob.size == len(dates):
        cp_prob = cp_prob[cp_index]  
    
    mask           = cp_prob >= mask_num
    cp_index_keep  = cp_index[mask].tolist()
    cp_prob_keep   = cp_prob[mask].tolist()
    cp_date_keep   = [dates[i] for i in cp_index_keep]
    print(f"[{h3_id}] cp_date_keep: {cp_date_keep}")

    def _py(v):
        import numpy as np
        if isinstance(v, np.generic):
            return v.item()
        if isinstance(v, np.ndarray):
            return v.tolist()
        return v
    
    return {k: _py(v) for k, v in {
        "id": h3_id,
        "trend": trend_vals,
        "fitted": fitted_vals,
        "cp_index": cp_index_keep,
        "cp_date":  [d.strftime("%Y-%m-%d") for d in cp_date_keep],
        "cp_prob":  cp_prob_keep,
        "r2": getattr(res, "R2", None),
        "rmse": getattr(res, "RMSE", None),
        "mape": getattr(res, "MAPE", None),
        "n_cp": len(cp_index_keep),
        "season_order": None,
        "freq": freq,
    }.items()}


#3. 串行处理，容错，中途储存


# 断点续联
completed = set()
if os.path.exists(out_trend_path):
    completed = set(pd.read_csv(out_trend_path, index_col=0).index)

for idx in tqdm(df.index, desc="Serial BEAST"):
    if idx in completed:
        continue  
    values = df.loc[idx].to_numpy()
    try:
        r = run_beast_on_series(idx, values, dates, freq)
        results.append(r)
        
        trend_series = pd.Series(
            data=r['trend'], 
            index=dates, 
            name=r['id']
        )
        trend_series.to_frame().T.to_csv(
            out_trend_path,
            mode='a',
            header=not os.path.exists(out_trend_path)
        )
        # 写入突变点
        for d, p in zip(r["cp_date"], r["cp_prob"]):
            cp_records.append({"H3_id": r["id"], "cp_date": d, "cp_prob": p})
        # 每跑 100 条写一次 cp_records
        if len(cp_records) >= 100:
            pd.DataFrame(cp_records).to_csv(out_cp_path, mode='a', index=False,
                                           header=not os.path.exists(out_cp_path))
            cp_records = []
    except Exception as e:
        print(f"!! H3 {idx} 出错，跳过：{e}")

# 4.把剩余的 cp_records 写入
if cp_records:
    pd.DataFrame(cp_records).to_csv(out_cp_path, mode='a', index=False,
                                   header=not os.path.exists(out_cp_path))
5
print("Done")
print(f"  趋势：{out_trend_path}")
print(f"  突变点：{out_cp_path}")


with open(json_out_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"模型完整参数输出至：{json_out_path}")


尝试将列名转换为日期时间对象...
转换前的列名 (前5个): ['2017152', '2017153', '2017154', '2017155', '2017156']
转换后的列名 (前5个，类型 <class 'pandas.core.indexes.datetimes.DatetimeIndex'>): [Timestamp('2017-06-01 00:00:00'), Timestamp('2017-06-02 00:00:00'), Timestamp('2017-06-03 00:00:00'), Timestamp('2017-06-04 00:00:00'), Timestamp('2017-06-05 00:00:00')]


Serial BEAST:   0%|          | 0/82 [00:00<?, ?it/s]


Model fitted:  Y =  trend + error 

INFO: To supress messages, set print_param/print_warning/print_progresss = 0 in beast() and beast_irreg()
INFO: To supress messages, Set extra.printParameter/printWarning/printProgress = 0 in beast123()
INFO: To supress all messaages, set quiet=1 in beast() and beast_irreg() or extra.quiet=1 in beast123()

#--------------------------------------------------#
#       Brief summary of Input Data                #
#--------------------------------------------------#
Data Dimension: One signal of length 212
IsOrdered     : Yes, ordered in time
IsRegular     : Yes, evenly spaced at interval of  1 (unknown unit)
hasSeasonCmpnt: False | no periodic or seasonal component. The model Y=Trend+Error is fitted.
HasOutlierCmpt: False | If true, Y=Trend+Outlier+Error (experimental) is fitted instead of Y=Trend+Error 
Detrend       : False | If true, remove a global trend component before running BEAST & add it back after BEAST
MissingValue  : NaN  flagged as missin

Serial BEAST:   1%|          | 1/82 [00:00<00:21,  3.79it/s]

|Progress:100.0% done[==============================================================]
[DEBUG] raw cp = [ 76.  80.  69.  17. 191. 103. 131.  21. 108. 137.] (<class 'numpy.ndarray'>)
[DEBUG] cp_array (raveled) = [ 76.  80.  69.  17. 191. 103. 131.  21. 108. 137.]
[DEBUG] cp_array_clean = [ 76.  80.  69.  17. 191. 103. 131.  21. 108. 137.]
[DEBUG] cp_index = [ 76  80  69  17 191 103 131  21 108 137]
[DEBUG] cpOccPr full = [0.         0.         0.         0.         0.00408333 0.00454167
 0.00466667 0.00608333 0.00529167 0.00691667 0.00770833 0.008
 0.00708333 0.00754167 0.00733333 0.00758333 0.06770834 0.00754167
 0.01058333 0.00645833 0.009      0.00808333 0.00825    0.00791667
 0.005125   0.00575    0.00654167 0.00495833 0.00508333 0.004875
 0.00483333 0.00483333 0.00608333 0.00445833 0.00504167 0.00383333
 0.00433333 0.00491667 0.00470833 0.00458333 0.00708333 0.00466667
 0.00641667 0.00691667 0.00591667 0.00558333 0.00458333 0.00704167
 0.00429167 0.00470833 0.00520833 0.00683333 0.0

...

\Progress:100.0% done[==============================================================]
[DEBUG] raw cp = [105. 155.  33. 131. 191. 172.  89.   9.  80.  17.] (<class 'numpy.ndarray'>)
[DEBUG] cp_array (raveled) = [105. 155.  33. 131. 191. 172.  89.   9.  80.  17.]
[DEBUG] cp_array_clean = [105. 155.  33. 131. 191. 172.  89.   9.  80.  17.]
[DEBUG] cp_index = [105 155  33 131 191 172  89   9  80  17]
[DEBUG] cpOccPr full = [0.         0.         0.         0.         0.00679167 0.00704167
 0.00691667 0.00775    0.03704167 0.00804167 0.00783333 0.0085
 0.00683333 0.008375   0.008875   0.00479167 0.03145834 0.00608333
 0.00620833 0.00566667 0.00658333 0.00654167 0.00775    0.00641667
 0.00583333 0.00654167 0.00708333 0.006875   0.008625   0.007125
 0.00729167 0.00866667 0.03266667 0.00958333 0.03054167 0.00966667
 0.00991667 0.009      0.00945834 0.00883333 0.01004167 0.00695833
 0.009      0.01041667 0.01170833 0.00795833 0.00770833 0.00683333
 0.00691667 0.007      0.007      0.007125   0.

Serial BEAST: 100%|██████████| 82/82 [00:18<00:00,  4.58it/s]


Model fitted:  Y =  trend + error 

INFO: To supress messages, set print_param/print_warning/print_progresss = 0 in beast() and beast_irreg()
INFO: To supress messages, Set extra.printParameter/printWarning/printProgress = 0 in beast123()
INFO: To supress all messaages, set quiet=1 in beast() and beast_irreg() or extra.quiet=1 in beast123()

#--------------------------------------------------#
#       Brief summary of Input Data                #
#--------------------------------------------------#
Data Dimension: One signal of length 212
IsOrdered     : Yes, ordered in time
IsRegular     : Yes, evenly spaced at interval of  1 (unknown unit)
hasSeasonCmpnt: False | no periodic or seasonal component. The model Y=Trend+Error is fitted.
HasOutlierCmpt: False | If true, Y=Trend+Outlier+Error (experimental) is fitted instead of Y=Trend+Error 
Detrend       : False | If true, remove a global trend component before running BEAST & add it back after BEAST
MissingValue  : NaN  flagged as missin

Serial BEAST: 100%|██████████| 82/82 [00:18<00:00,  4.38it/s]

Done
  趋势：H:\nighttimeLights\south_2017tif_layers\beast\beast_trend_12021.csv
  突变点：H:\nighttimeLights\south_2017tif_layers\beast\beast_change_points_12021.csv
模型完整参数输出至：H:\nighttimeLights\south_2017tif_layers\beast\beast_model_outputs_12021.json


In [30]:
wide_data_avrg

NameError: name 'wide_data_avrg' is not defined